# How to use Cloudflare D1 checkpointer for persistence

Prerequisites

The guide assumes familiary with the following:

* [Persistence](https://langchain-ai.github.io/langgraph/concepts/persistence/)
* [Cloudflare D1](https://developers.cloudflare.com/d1/)

When creating LangGraph agents, you can also set them up so that they persist their state. This allows you to do things like interact with an agent multiple times and have it remember previous interactions.

This reference implementation shows how to use Cloudflare's D1 serverless database as the backend for persisting checkpoint state using the `langgraph-checkpoint-cloudflare-d1` library.

For demonstration purposes we add persistence to a [prebuilt ReAct agent](https://langchain-ai.github.io/langgraph/agents/agents/#basic-configuration).

## Setup

To use the Cloudflare D1 checkpointer, you will need a Cloudflare D1 database. Follow this [guide](https://developers.cloudflare.com/d1/get-started/) to create a database if you don't already have one.

Next, let's install the required packages and set our API keys

In [ ]:
%%capture --no-stderr
%pip install -U requests httpx langgraph langgraph-checkpoint-cloudflare-d1

In [ ]:
import getpass
import os

if not os.getenv("CF_API_KEY"):
    os.environ["CF_API_KEY"] = getpass.getpass("Enter your CloudflareWorkersAI API key: ")

if not os.getenv("CF_ACCOUNT_ID"):
    os.environ["CF_ACCOUNT_ID"] = getpass.getpass("Enter your Cloudflare account ID: ")

if not os.getenv("CF_D1_API_TOKEN"):
    os.environ["CF_D1_API_TOKEN"] = getpass.getpass("Enter your Cloudflare D1 API token: ")

If you'd like to use Cloudflare's WorkersAI, use the following:

In [ ]:
%pip install -qU langchain-cloudflare

In [ ]:
if not os.getenv("CF_API_KEY"):
    os.environ["CF_API_KEY"] = getpass.getpass("Enter your CloudflareWorkersAI API key: ")

In [6]:
from langchain_cloudflare import ChatCloudflareWorkersAI

model = ChatCloudflareWorkersAI(
    model="@cf/meta/llama-3.3-70b-instruct-fp8-fast",
    temperature=0,
    max_tokens=1024
)

Otherwise you can use another LLM provider like below

In [ ]:
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI API Key: ")

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
)

Set up [LangSmith](https://smith.langchain.com/) for LangGraph development

Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started [here](https://docs.smith.langchain.com/).

## Define model and tools for the graph

In [7]:
from typing import Literal

from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]

# Cloudflare D1 checkpointer usage

In [10]:
from langgraph_checkpoint_cloudflare_d1 import CloudflareD1Saver

checkpointer = CloudflareD1Saver()

graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
config = {"configurable": {"thread_id": "1"}}
response = graph.invoke(
    {"messages": [("human", "what's the weather in sf")]}, config
)

## Async

In [14]:
from langgraph_checkpoint_cloudflare_d1 import AsyncCloudflareD1Saver

checkpointer = AsyncCloudflareD1Saver()

graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
config = {"configurable": {"thread_id": "2"}}
response = await graph.ainvoke(
    {"messages": [("human", "what's the weather in sf")]}, config
)